In [1]:
#%% make parent tree graph
from treelib import Tree
import numpy as np
import os 
import pandas as pd
import requests
from math import ceil
from os.path import join as pjoin
from PIL import (
    Image,
    ImageFont,
    ImageDraw,
)

utils_path = '/nfs/z1/userhome/ZhouMing/workingdir/BIN/action/utils/HAD/data/utils'
result_path = '/nfs/z1/userhome/ZhouMing/workingdir/BIN/action/utils/HAD/result'
taxonomy = pd.read_csv(f'{utils_path}/taxonomy_raw.csv')

In [5]:
# prepare selected classes
action_dataset_path = '/nfs/z1/userhome/ZhouMing/workingdir/BIN/action/exp/video'
class_selected = os.listdir(action_dataset_path)
class_all = pd.read_csv(f'{utils_path}/taxonomy.csv')['nodeName'].tolist()
class_remove = list(set(class_all).difference(set(class_selected)))
class_remove.extend(['Health-related Self Care', 'Health-related self care'])

remain_node = {'nodeName':[], 'nodeId':[], 'parentId':[]}
superclass_names = ['Household Activities', 'Personal Care', 'Eating and drinking Activities', 
                    'Sports, Exercise, and Recreation', 'Socializing, Relaxing, and Leisure']
superclass_ids = [1, 2, 4, 5, 7]                
remove_id = []
tree = Tree()
select_class_shown = 0

for row in range(taxonomy.shape[0]):
    nodeName, nodeId, parentId = taxonomy.loc[row, ['nodeName', 'nodeId', 'parentId']]
    if not np.isnan(parentId):
        parentId = int(parentId)
    else:
        parentId = None
    # record remove ids
    if nodeName in class_remove:
        remove_id.append(nodeId)

    # calculate how many class has been shown in the tree
    if nodeName in class_selected:
        select_class_shown += 1

    # adding node into tree
    try:
        tree.create_node(tag=nodeName, identifier=nodeId, parent=parentId)
    except:
        # if parentId is not defined, then add it into dict
        remain_node['nodeName'].append(nodeName)
        remain_node['nodeId'].append(nodeId)
        remain_node['parentId'].append(parentId)
        continue

# Add undefined parentId into tree
for idx in range(len(remain_node['nodeName'])):
    nodeName = remain_node['nodeName'][idx]
    nodeId = remain_node['nodeId'][idx]
    parentId = remain_node['parentId'][idx]
    tree.create_node(tag=nodeName, identifier=nodeId, parent=parentId)

# Remove unwanted classes
for idx in remove_id:
    tree.remove_node(idx)

print('Already match %d classes'%(select_class_shown))

# get node number info
for idx, node_id in enumerate(superclass_ids):
    sub_tree = tree.subtree(node_id)
    if node_id == 2:
        sub_tree.save2file(f'{result_path}/hierarchy/{superclass_names[idx]}.txt')

Already match 180 classes


In [33]:
PIL_WIDTH_INDEX = 0
PIL_HEIGHT_INDEX = 1
font_file = '/nfs/z1/userhome/ZhouMing/workingdir/BIN/utils/font/STIXGeneral.ttf'

def textfile_to_image(textfile_path, font_color):
    """Convert text file to a grayscale image.

    arguments:
    textfile_path - the content of this file will be converted to an image
    font_path - path to a font file (for example impact.ttf)
    """
    # parse the file into lines stripped of whitespace on the right side
    with open(textfile_path, encoding='utf-8') as f:
        lines = tuple(line.rstrip() for line in f.readlines())

    # choose a font (you can see more detail in the linked library on github)
    font = None
    large_font = 27  # get better resolution with larger size
    font = ImageFont.truetype(font_file, size=large_font)
    if font is None:
        font = ImageFont.load_default()
        print('Using default font.')

    # make a sufficiently sized background image based on the combination of font and lines
    font_points_to_pixels = lambda pt: round(pt * 96.0 / 72)
    margin_pixels = 1

    # # height of the background image
    # tallest_line = max(lines, key=lambda line: font.getsize(line)[PIL_HEIGHT_INDEX])
    # max_line_height = font_points_to_pixels(font.getsize(tallest_line)[PIL_HEIGHT_INDEX])
    # realistic_line_height = max_line_height * 0.8  # apparently it measures a lot of space above visible content
    # image_height = int(ceil(realistic_line_height * len(lines) + 2 * margin_pixels))
    # print(realistic_line_height)
    # # width of the background image
    # widest_line = max(lines, key=lambda s: font.getsize(s)[PIL_WIDTH_INDEX])
    # max_line_width = font_points_to_pixels(font.getsize(widest_line)[PIL_WIDTH_INDEX])
    # image_width = int(ceil(max_line_width + (2 * margin_pixels)))

    image_height, image_width = 3000, 612
    realistic_line_height = 28

    # draw the background
    background_color = (255, 255, 255)  # white
    image = Image.new("RGB", (image_width, image_height), color=background_color)
    draw = ImageDraw.Draw(image)

    # draw each line of text
    horizontal_position = margin_pixels
    for i, line in enumerate(lines):
        vertical_position = int(round(margin_pixels + (i * realistic_line_height)))
        draw.text((horizontal_position, vertical_position), line, fill=font_color, font=font)

    return image

colors = ['blue', '#0ac4d1', '#FF0032', '#A31ACB', 'green']
txt_path = f'{result_path}/hierarchy'
txt_file = [x for x in os.listdir(txt_path) if x.endswith('.txt')]
for idx, file in enumerate(txt_file):
    print('%s_%s'%(file, colors[idx]))
    image = textfile_to_image(pjoin(txt_path, file), colors[idx])
    image.save(pjoin(txt_path, f"{file.split('.')[0]}.jpg"))

Socializing, Relaxing, and Leisure.txt_blue
Eating and drinking Activities.txt_#0ac4d1
Sports, Exercise, and Recreation.txt_#FF0032
Household Activities.txt_#A31ACB
Personal Care.txt_green
